<a href="https://colab.research.google.com/github/MohitDhungana/duplicate_question_detection/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np

import re

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
def text_to_word_list(text):
#     gets a whole question inside text variable on which  preprocessing is done and then the question is splitted into word indices and returned
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text
  
  


In [0]:
# sample question
ques1='How do I read and find my YouTube comments?'
ques2='comments YouTube my find and read I do How?'

In [0]:
# import json file from drive and parse in python 

import json

dict_path='/content/drive/My Drive/vocabulary_dictionary.json'

# read file
with open(dict_path, 'r') as my_file:
    data=my_file.read()

# parse file
vocab_dictionary = json.loads(data)

In [57]:
ques1_index=[]
for word in text_to_word_list(ques1):
  if word in vocab_dictionary:
    ques1_index.append(vocab_dictionary[word])
ques1_index

[26, 97, 16, 125, 48, 30, 126, 127]

In [0]:
# function to index tokenized words using training vocabulary dictionary
def word_indexer(question):
  ques_index=[]

  for word in text_to_word_list(question):
    if word in vocab_dictionary:
      ques_index.append(vocab_dictionary[word])      
  return ques_index

In [59]:
# preprocess and tokenize sample questions 
q1=word_indexer(ques1)
print(q1)
q2=word_indexer(ques2)
print(q2)

[26, 97, 16, 125, 48, 30, 126, 127]
[127, 126, 30, 48, 125, 16, 97, 26]


In [60]:
# zero padding above tokenized indices

from tensorflow.keras.preprocessing.sequence import pad_sequences

question_list=[q1,q2]
question_list=pad_sequences(question_list,maxlen = 50)
print(question_list)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  26  97  16 125  48  30 126 127]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 127 126  30  48 125  16  97  26]]


In [0]:
# manhattan distance calculation function
import tensorflow.keras.backend as K
def manhattan_distance(left,right):
  return K.exp(-K.sum(K.abs(left-right),axis=1, keepdims=True))

In [62]:
print(question_list[0].shape)
print(question_list[1].shape)

(50,)
(50,)


In [63]:
# Recreate the exact same model, including weights and optimizer.
import tensorflow as tf
from tensorflow.keras.models import load_model

model_path='/content/drive/My Drive/dqd_model_weights.h5'
new_model = load_model(model_path)
new_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
right_input (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 50, 300)      25796400    left_input[0][0]                 
                                                                 right_input[0][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 40)           51520       embedding_layer[0][0]      

In [64]:
# predict sample questions on saved model

# loss, acc = new_model.predict([question_list[0],question_list[1]])
# # print("Restored model, accuracy: {:5.2f}%".format(100*acc))

pred=new_model.predict([question_list[0],question_list[1]])
print(pred)

ValueError: ignored